# Atmospheric pressure correction

This notebook uses example data to generate an atmospheric correction file.

In [1]:
from __future__ import division, print_function
import stglib
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

filename = 'atmpressure.csv'
cdfRawFile = '../StglibProcessing/051161_B2-raw.cdf'
ncfile1 = 'temp.nc'

## Generate atmospheric .nc file

In [2]:
def read_met_data(filename):
    a = pd.read_csv(filename, header=0, parse_dates=[0],
                    infer_datetime_format=True, index_col=0)

    return xr.Dataset(a)

gndcrmet = read_met_data(filename) # This creates an xarray Dataset
gndcrmet = gndcrmet['pres'].to_dataset() # Let's keep only the BP variable
gndcrmet['pres'] = gndcrmet['pres']/100 # convert our atmos data (in millibars) to decibars
gndcrmet.to_netcdf(ncfile1) # This saves to a .nc file. Not required here as we will just be reading it back again

## Generate the atmpres.cdf file 
This generates the file and embeds the instrument-specific offset as an attr. The trickiest part of this process is determining what to use as an offset. After you run this cell, you will have your very own atmpres.cdf file!

In [3]:
# Load the raw Aquadopp data
RAW = xr.open_dataset(cdfRawFile, autoclose=True)

# Load the met data
gndcrmet = xr.open_dataset(ncfile1, autoclose=True)

met = gndcrmet['pres'] # make a new met variable
met = met.rename('atmpres') # rename it to the standard atmpres variable name
met = met.reindex(time=RAW['time'], copy=True, method='nearest') # reindex the met data onto the Aquadopp time base
met.attrs.update(offset=0) #-10.15) # set the atmospheric offset as an attribute
met.to_netcdf('atmpres.cdf') # save to disk

C:\Users\Crosby\AppData\Local\conda\conda\envs\rskprocessing\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in xarray v0.12. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  
C:\Users\Crosby\AppData\Local\conda\conda\envs\rskprocessing\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in xarray v0.12. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  """
C:\Users\Crosby\AppData\Local\conda\conda\envs\rskprocessing\lib\site-packages\xarray\core\dataarray.py:1748: SerializationWarning: saving variable time with floating point data as an integer dtype without any _FillValue to use for NaNs
  return dataset.to_ne

## Load clean data
Note that you need to run the proper run scripts with your generated atmpres.cdf files... this only uses example files.

In [ ]:
def load_clean(filename, basedir):
    fildir = basedir

    ds = xr.open_dataset(basedir + filename, decode_times=False, autoclose=True)
    ds['time'] = ds['time_cf']
    ds = ds.drop('time2')
    
    return xr.decode_cf(ds)

VEL = load_clean('10761Aaqd-a.nc', basedir)

## View data
See how the raw and P_1ac data compare.

In [ ]:
plt.figure(figsize=(10,8))
RAW['Pressure'].plot()
VEL['P_1ac'].plot()
plt.show()